# Basketball (or Netball) Cannon Toy Problem

Develop a controller for a basketball cannon.  Given the following inputs
1. distance from the basket
2. height from which the ball leaves the cannon
3. speed the ball will leave the cannon

**What angle should the cannon aim the ball such that it will go through the basket without interacting with the backboard or the rim.**

Note that this scenario in actually more similar to Netball than Basketball, since we are neglecting the backboard.
<img src="https://netballamerica.com/wp-content/uploads/1DX28219.jpg">

In [1]:
# import some basic packages
import itertools
import numpy as np
from collections import defaultdict
from typing import Tuple

In [2]:
np.set_printoptions(suppress=True)

In [3]:
from bokeh import plotting
from bokeh.palettes import Dark2_8 as palette
colors = itertools.cycle(palette)

In [4]:
plotting.output_notebook()

Loading BokehJS ...

In this problem, we observe that the parameters are such that cannon is always aimed in the vertical plane connecting the cannon and the hoop. We can separate the motion of the ball into the horizontal, $x$, and vertical, $y$, directions.  The position in both directions can be written using the linear equations of motions as follows.

$$x(t) = x_0 + \dot{x}_0 t + \frac{1}{2} \ddot{x}_0 t^2$$

$$y(t) = y_0 + \dot{y}_0 t + \frac{1}{2} \ddot{y}_0 t^2$$

We can set the x and y origin at the base of the cannon, so the ball leaves the cannon at $(x_0, y_0) = (0, h_0)$.  The speed at which the ball leaves the cannon is written in terms of x and y as $(\dot{x}_0, \dot{y}_0) = (v\cos\theta, v\sin\theta)$.  The only acceleration in this situation is gravity so $(\ddot{x}_0, \ddot{y}_0) = (0, g)$.  Based on these values, the equations of motion can be written as

$$x(t) = v\cos(\theta) t$$

$$y(t) = h_0 + v\sin(\theta) t + \frac{1}{2}g t^2$$


To measure how close each shot gets to the basket, we want to consider the time when the ball passes the height of the basket, $h_{b}$, for the second time.  

$$\frac{1}{2} g t^2 + v \sin(\theta) t + \delta_h = 0$$

where $\delta_h$ is the difference in height, $\delta_h = h_0 - h_{basket}$.  This occurs when 

$$t = \frac{-v \sin(\theta) \pm \sqrt{v^2 \sin^2(\theta)+2gh}}{g}$$

Note that in order to have a reasonable throw, using the assumption that the cannon is always below the rim, the trajectory of the ball must form an arch passing height of the rim twice.  This only occurs when the descriminant is positive, which means

$$v^2 > \frac{-2 g h}{sin^2(\theta)}$$

Lets begin by defining a cannon class that expects the three inputs, distance, height, and speed.  We also define the `fire` method which takes an angle as input then provides as output the distance and impact angle the ball will make with the horizontal plane of the hoop.  We also define a `show_path` method which plots all `fire` attempts.

Note that we are using a ball width of 24 cm, and a rim that is twice that width.  This means that the ball goes through the rim if the center of the ball is within half the ball width of the center of the rim.

In [5]:
class Cannon():
    def __init__(self, dist: float, height: float, speed: float) -> None: 
        # constants
        self.ball_radius = 0.24 / 2 # m
        self.height_basket = 3.05  # m
        self.g = -9.8  # m/s^2
        
        # cannon parameters
        self.dist = dist  # m, distance to hoop
        self.height = height  # m, height of the cannon
        self.speed = speed  # m/s
        
        self.delta_h = self.height_basket - self.height
        self.rim = dist + self.ball_radius * np.array([-1, 1])  # m

        self.path = []
        self.angle = 0
        
    def fire(self, angle: float):
        self.angle = np.deg2rad(angle)
        cos = np.cos(self.angle)
        sin = np.sin(self.angle)

        self.result = 'terrible shot'
        D = (self.speed * sin) ** 2 + 2 * self.g * self.delta_h
        if D > 0:  # ball passes through correct height twice
            self.t_m = (-self.speed * sin - np.sqrt(D)) / self.g  # negative root is the downward part of the arc
            self.x_shot = self.speed * cos * self.t_m  
            self.delta_x = self.x_shot - self.dist  # neg: undershot, pos: overshot
            if self.rim[0] < self.x_shot < self.rim[1]:
                self.result = 'success'
            else:
                self.result = 'miss'
            t_final = self.t_m * 1.1
        else:
            # TODO: consider this case further
            self.delta_x = np.nan  # what's a better way to treat this?
            t_final = 1.5  # s

        t = np.linspace(0, t_final, 100)  # 0 to 10 s
        x = self.speed * cos * t
        y = self.height + self.speed * sin * t + self.g * t**2 / 2
        self.path.append(np.c_[x, y][y > 0])
        
        return self.delta_x
    
    def _dx(self, angle):
        sin = np.sin(angle)
        cos = np.cos(angle)
        D = (self.speed * sin) ** 2 + 2 * self.g * self.delta_h
        sqrt_D = np.sqrt(D)
        dx = (self.speed ** 2 / self.g * (sin ** 2 - cos ** 2) 
              - self.speed * sin / self.g * (-sqrt_D + (self.speed * cos) ** 2 / sqrt_D))
        return dx        
        
    def show_path(self):
        if self.path:
            p = plotting.figure(
                width=800, height=400, 
                x_axis_label='(meters)', y_axis_label='(meters)',
                x_range=(-1, self.dist*1.3), y_range=(-0.5, 6),
            )

            # plot the hoop
            p.line(np.array([1, 1.015, 1.015]) * self.rim[1], 
                   [self.height_basket, self.height_basket, 0], color='gray')
            p.circle(self.rim, self.height_basket,  
                     radius=0.02, color='orange')
            p.line(self.rim, self.height_basket, color='orange')

            # plot the attempts
            for i, (path, color) in enumerate(zip(self.path, colors)):
                name = f'Attempt {i+1}'
                p.line(path[:, 0], path[:, 1], color=color, 
                       legend_label=name, muted_alpha=0.2)
                t = np.linspace(0, 1, 100)
                
            # plot the last cannon
            p.line(0, [0, self.height], color='black')
            p.line(
                [-np.cos(self.angle)/2, 0], 
                [self.height - np.sin(self.angle)/2, self.height],
                line_width=5, color=color,
            )

            p.legend.location = 'top_right'
            p.legend.click_policy = 'mute'
            p.toolbar.autohide = True
            plotting.show(p)
        else:
            print('Error: no attempts made!')

Lets look at a cannon placed roughly at the free-throw line, 5 meters from the basket, shot from 2 meters above the ground, and fired at 8 meters per second.

In [6]:
freethrow = Cannon(5, 2, 8)

for angle in [45, 40, 43, 60]:
    delta_x = freethrow.fire(angle)
    print(f'{angle} degrees: {freethrow.result} ({delta_x:0.2f} m away)')

freethrow.show_path()

45 degrees: miss (0.22 m away)
40 degrees: miss (-0.27 m away)
43 degrees: success (0.07 m away)
60 degrees: success (-0.03 m away)


We see that the first shot, at an angle of 45 degrees, was 0.22 meters past the basket.
The second shot, at an angle of 40 degrees, was 0.27 meters short of the basket.
The third shot, at 43 degrees went through the basket.
Maybe surprising, the fourth shot, at 60 degrees, also went through the basket.

Lets position the cannon roughtly at the three-point line, 7 meters from the basket.  We will use the same height, but increase the velocity a bit as we are a bit farther away.

In [7]:
three_point = Cannon(7, 2, 9)

three_point.fire(45)  # Rachael's first attempt
three_point.fire(54)  # 2nd angle
three_point.show_path()

How do we figure out the angle for other situations?  We could always brute force a solution.

In [8]:
c3 = Cannon(15, 2, 15)
res = []
for angle in range(20, 75):
    res.append(c3.fire(angle))

c3.show_path()

## Learn what angle to fire the cannon

A more interesting approach is to create an agent that can explore firing this cannon to experimentally learn what angle to aim the cannon to successfully fire the ball into the hoop.

The goal for our agent, or cost function we want to minimize, is the distance the ball passes from the center of the basket (`delta_x`).

In [9]:
lr = [1]
dn = 0.01
for i in range(50):
    lr.append(lr[-1]/(1 + dn))

In [10]:
class Agent():
    def __init__(self, cannon, learning_rate, decay_rate):
        self.cannon = cannon
        self.lr = learning_rate
        self.rng = np.random.default_rng()
        self.decay_rate = decay_rate
    
    def _step(self, i):
        # this is not the best as it is only one-sided
        d_delta_x = np.abs(self.results[i, 1]) - np.abs(self.results[i-1, 1])
        d_angle = self.results[i, 0] - self.results[i-1, 0]
        self.lr /= 1 + self.decay_rate
        step = self.lr * d_delta_x / d_angle
        self.results[i+1, 0] = self.results[i, 0] - step        
    
    def learn(self, max_iters, *, angle_0=89):
        # hacky gradient descent
        
        self.results = np.empty((max_iters, 2))
        
        delta_x_0 = self.cannon.fire(angle_0)
        self.results[0, :] = angle_0, delta_x_0
        
        if angle_0 < 45:
            angle_1 = angle_0 + 1
        else:
            angle_1 = angle_0 -1
            
        delta_x_1 = self.cannon.fire(angle_1)
        self.results[1, :] = angle_1, delta_x_1
        self._step(1)

        
        for i in range(2, max_iters-1):
            self.results[i, 1] = self.cannon.fire(self.results[i, 0])
            if np.abs(self.results[i, 1]) < self.cannon.ball_radius:
                break
            self._step(i)
        
        self.results = self.results[:i+1]

Test the agent with a cannon that is 15 meters away, 2 meters off the ground, and firing at a speed of 15 meters / second.

Use an initial guess of 20 degrees.

In [11]:
c3 = Cannon(15, 2, 15)
learning_rate = 0.8
decay_rate = 0.01
johnny5 = Agent(c3, learning_rate, decay_rate)
johnny5.learn(100, angle_0=20)

In [12]:
johnny5.results

array([[20.        , -4.17512663],
       [21.        , -3.19787242],
       [21.77406274, -2.51138164],
       [22.46957659, -1.9336059 ],
       [23.11460581, -1.42527077],
       [23.72046991, -0.96888178],
       [24.2938506 , -0.55393602],
       [24.83924417, -0.17337444],
       [25.35990548,  0.17788031],
       [25.35351191,  0.17363579],
       [24.86790669, -0.15373799],
       [24.83823118, -0.17406908],
       [25.32949801,  0.15767855],
       [25.35318495,  0.17341869],
       [24.88608304, -0.14130391],
       [24.83823282, -0.17406795],
       [25.31005991,  0.14474451],
       [25.3524613 ,  0.17293816],
       [24.90330469, -0.12953608],
       [24.83867709, -0.17376329],
       [25.29184118,  0.13260766],
       [25.35138495,  0.17222338],
       [24.91949569, -0.11848414]])

In [13]:
c3.show_path()

In [14]:
len(johnny5.results)

23

The agent found the correct angle in 23 trials, with several oscillations around the correct answer.  This means our initial learning rate is too large, or our decay rate is too small.  Looking at the initial step, they are never more than a degreee, se we could reasonably increase the initial learning rate.  

Lets increase both the learning rate and the decay rate. 

In [15]:
c3 = Cannon(15, 2, 15)
learning_rate = 2
decay_rate = 0.1
johnny5 = Agent(c3, learning_rate, decay_rate)
johnny5.learn(100, angle_0=20)

In [16]:
johnny5.results

array([[20.        , -4.17512663],
       [21.        , -3.19787242],
       [22.77682584, -1.6884076 ],
       [24.18100577, -0.63436227],
       [25.30895218,  0.14400696],
       [25.9028089 ,  0.53226298],
       [25.09090786, -0.00216573]])

In [17]:
c3.show_path()

In [18]:
len(johnny5.results)

7

Now it found a valid angle in only 7 steps with the initial steps being a little less than 2 degrees.


Lets use the same cannon with a initial angle of 80 degrees.

In [19]:
c3 = Cannon(15, 2, 15)
learning_rate = 2
decay_rate = 0.1
johnny5 = Agent(c3, learning_rate, decay_rate)
johnny5.learn(100, angle_0=80)

In [20]:
johnny5.results

array([[80.        , -7.33722412],
       [79.        , -6.60852564],
       [77.67509368, -5.65957943],
       [76.49123177, -4.82915605],
       [75.43720786, -4.10496123],
       [74.49864318, -3.47297971],
       [73.66244977, -2.92075704],
       [72.91689187, -2.43741374],
       [72.25153365, -2.01353989],
       [71.65714622, -1.64104723],
       [71.12559646, -1.31300874],
       [70.64973117, -1.02350216],
       [70.22326472, -0.76746611],
       [69.84067436, -0.54057217],
       [69.49710538, -0.33911372],
       [69.18828642, -0.1599106 ],
       [68.91045501, -0.00022826]])

In [21]:
c3.show_path()

This seems reasonable.  What if we start by overshooting the basket.

In [22]:
c3 = Cannon(15, 2, 15)
learning_rate = 2
decay_rate = 0.01
johnny5 = Agent(c3, learning_rate, decay_rate)
johnny5.learn(1000, angle_0=50)

In [23]:
johnny5.results

array([[50.        ,  6.69202709],
       [49.        ,  6.78307679],
       [49.18029643,  6.76879035],
       [49.33565099,  6.75572756],
       [49.49887265,  6.74125368],
       [49.66930488,  6.72532173],
       [49.8471903 ,  6.70780231],
       [50.0327489 ,  6.68855913],
       [50.22620218,  6.66744647],
       [50.42777213,  6.64430885],
       [50.63768087,  6.61898062],
       [50.85615025,  6.5912856 ],
       [51.0834014 ,  6.56103669],
       [51.3196543 ,  6.52803557],
       [51.56512727,  6.49207236],
       [51.82003648,  6.45292536],
       [52.08459538,  6.41036079],
       [52.35901418,  6.36413262],
       [52.64349921,  6.31398244],
       [52.93825233,  6.25963933],
       [53.24347022,  6.20081991],
       [53.55934377,  6.13722838],
       [53.88605731,  6.06855669],
       [54.22378791,  5.99448475],
       [54.57270455,  5.91468085],
       [54.93296739,  5.82880206],
       [55.3047269 ,  5.73649487],
       [55.68812302,  5.63739589],
       [56.08328431,

In [24]:
c3.show_path()

The previous decay rate of 0.1 is too agressive in this case, halting the progress before it reaches the minimum.  Changing to a decay rate of 0.01 resolves this issue.

In [25]:
c3 = Cannon(15, 2, 15)
learning_rate = 2
decay_rate = 0.05
johnny5 = Agent(c3, learning_rate, decay_rate)
johnny5.learn(100, angle_0=40)

In [26]:
johnny5.results

array([[40.        ,  6.28086346],
       [41.        ,  6.45580332],
       [40.66678123,  6.40084924],
       [40.36760861,  6.3486675 ],
       [40.0662672 ,  6.29338661],
       [39.7644187 ,  6.23527313],
       [39.46272145,  6.17444701],
       [39.16182785,  6.11105086],
       [38.86235678,  6.0452434 ],
       [38.56489181,  5.97719739],
       [38.26997934,  5.90709727],
       [37.97812738,  5.83513685],
       [37.6898047 ,  5.76151695],
       [37.40544057,  5.68644319],
       [37.12542481,  5.61012377],
       [36.85010823,  5.53276747],
       [36.57980338,  5.45458173],
       [36.31478562,  5.37577091],
       [36.05529434,  5.29653479],
       [35.80153441,  5.21706715],
       [35.55367777,  5.13755463],
       [35.31186515,  5.05817572],
       [35.07620785,  4.97909993],
       [34.84678954,  4.90048716],
       [34.6236682 ,  4.82248724],
       [34.40687789,  4.74523955],
       [34.19643069,  4.66887285],
       [33.99231843,  4.5935052 ],
       [33.79451454,

In [27]:
c3.show_path()

In [28]:
c3 = Cannon(15, 2, 15)
johnny5 = Agent(c3, learning_rate=1)
johnny5.learn(100, angle_0=80)

TypeError: __init__() missing 1 required positional argument: 'decay_rate'

In [ ]:
johnny5.results

In [ ]:
c3.show_path()

In [ ]:
len(johnny5.results)

Now it found a valid angle in only 17 steps.


What if we start by overshooting?

In [ ]:
c3 = Cannon(15, 2, 15)
johnny5 = Agent(c3, learning_rate=1)
johnny5.learn(100, angle_0=45)

In [ ]:
johnny5.results

In [ ]:
c3.show_path()

This shows we were able to get to both successful angles starting from the scenario of both under and over shooting.  Further exploration could be given to tuning the learning rate and decay rate.  Also, this solution will only work for angle between 0 and 90 degrees.  We did not consider the possibility of the cannon being higher than the basket, in which case angles below 0 degrees are viable options.  There are certainly several alternative approaches that could be taken, one of which would be reinforcement learning, defining a punishment based on how far each shot is from the basket, and a reward for successfully shooting the ball the basket.